## 문제 정의
- 환자의 데이터를 입력으로 하여 갑상선 질환 여부를 분류하는 이진 분류 문제
    - 평가지표: F1-score
- 컴피티션 주소
    - https://www.kaggle.com/competitions/scu-ai-competition-202401



In [58]:
# from google.colab import drive
# drive.mount('/content/drive')

# 컴피티션 데이터 다운로드 및 압축 풀기

In [59]:
# !gdown 1IOgB_HQs0BrXsD4DssDnHAlsmtmWCEN7 # 데이터 압축 파일 다운로드
# !unzip -oqq scu-ai-competition-202401.zip # 압축 풀기

- 데이터 경로 변수


In [60]:
DATA_PATH = "C:/Users/ltk65/Downloads/scu-ai-competition-202401/"
DATA_PATH

'C:/Users/ltk65/Downloads/scu-ai-competition-202401/'

- 시드값

In [61]:
SEED = 42

- 데이터 불러오기

In [62]:
import pandas as pd
import numpy as np

train = pd.read_csv(f"{DATA_PATH}train.csv") # 학습데이터
test = pd.read_csv(f"{DATA_PATH}test.csv") # 테스트 데이터
train.shape , test.shape

((4223, 18), (3456, 17))

# 타겟 컬럼
- 0: 정상 환자
- 1: 갑상선 질환 환자

In [63]:
train.head()

,ID,나이,성별,티록신_복용_여부,항갑상선제_복용_여부,지병_여부,임신_여부,갑상선_수술_이력,I131_치료_여부,갑상선저하_인지_여부,갑상선항진증_인지_여부,리튬_치료_여부,갑상선종_여부,종양_여부,TSH,FreeT3,FreeT4,target
0,train_0,59.0,남,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,NaN,NaN,0.77,0
1,train_1,63.0,남,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,33.0,1.5,NaN,1
2,train_2,65.0,여,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,1.7,2.3,0.95,0
3,train_3,33.0,남,아니오,아니오,예,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,6.2,NaN,0.66,0
4,train_4,64.0,여,예,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,1.2,NaN,0.95,0


In [64]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4223 entries, 0 to 4222
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            4223 non-null   object 
 1   나이            4220 non-null   float64
 2   성별            4086 non-null   object 
 3   티록신_복용_여부     4223 non-null   object 
 4   항갑상선제_복용_여부   4223 non-null   object 
 5   지병_여부         4223 non-null   object 
 6   임신_여부         4223 non-null   object 
 7   갑상선_수술_이력     4223 non-null   object 
 8   I131_치료_여부    4223 non-null   object 
 9   갑상선저하_인지_여부   4223 non-null   object 
 10  갑상선항진증_인지_여부  4223 non-null   object 
 11  리튬_치료_여부      4223 non-null   object 
 12  갑상선종_여부       4223 non-null   object 
 13  종양_여부         4223 non-null   object 
 14  TSH           3832 non-null   float64
 15  FreeT3        2989 non-null   float64
 16  FreeT4        3854 non-null   float64
 17  target        4223 non-null   int64  
dtypes: float64(4), int64(1), obj

# 타겟값 확인
- 비율 확인하기

In [65]:
train["target"].mean()

0.11816244376035993

# 결측치 처리

In [66]:
train.isnull().sum()

ID                 0
나이                 3
성별               137
티록신_복용_여부          0
항갑상선제_복용_여부        0
지병_여부              0
임신_여부              0
갑상선_수술_이력          0
I131_치료_여부         0
갑상선저하_인지_여부        0
갑상선항진증_인지_여부       0
리튬_치료_여부           0
갑상선종_여부            0
종양_여부              0
TSH              391
FreeT3          1234
FreeT4           369
target             0
dtype: int64

In [67]:
test.isnull().sum()

ID                0
나이                1
성별              117
티록신_복용_여부         0
항갑상선제_복용_여부       0
지병_여부             0
임신_여부             0
갑상선_수술_이력         0
I131_치료_여부        0
갑상선저하_인지_여부       0
갑상선항진증_인지_여부      0
리튬_치료_여부          0
갑상선종_여부           0
종양_여부             0
TSH             333
FreeT3          975
FreeT4          312
dtype: int64

- Data leakage를 피하기 위해 임의로 정한 상수 또는 학습데이터의 통계치를 이용하여 결측치를 처리해야 한다.

In [68]:
fill_age = train["나이"].mean()
fill_tsh = train["TSH"].mean()
fill_free_t3 = train["FreeT3"].mean()
fill_free_t4 = train["FreeT4"].mean()

In [69]:
train["나이"] = train["나이"].fillna(fill_age)
train["성별"] = train["성별"].fillna("UNK")
train["TSH"] = train["TSH"].fillna(fill_tsh)
train["FreeT3"] = train["FreeT3"].fillna(fill_free_t3)
train["FreeT4"] = train["FreeT4"].fillna(fill_free_t4)

In [70]:
test["나이"] = test["나이"].fillna(fill_age)
test["성별"] = test["성별"].fillna("UNK")
test["TSH"] = test["TSH"].fillna(fill_tsh)
test["FreeT3"] = test["FreeT3"].fillna(fill_free_t3)
test["FreeT4"] = test["FreeT4"].fillna(fill_free_t4)

In [71]:
train.isnull().sum().sum() , test.isnull().sum().sum()

(0, 0)

In [72]:
# train에 TSH 컬럼 나누기
def categorize_tsh(tsh_value):
    if tsh_value > 4.0:
        return 'High'
    else:
        return 'low'

train['TSH범위'] = train['TSH'].apply(categorize_tsh)

# test에 TSH 컬럼 나누기
def categorize_tsh(tsh_value):
    if tsh_value > 4.0:
        return 'High'
    else:
        return 'low'

test['TSH범위'] = test['TSH'].apply(categorize_tsh)

In [73]:
# train에 갑상선저하인지, 갑상선항진증인지 합쳐서 [정상,갑상선비정상] 으로 나누기
def determine_thyroid_status(row):
    if row['갑상선저하_인지_여부'] == "예":
        return 'hypo'
    elif row['갑상선항진증_인지_여부'] == '예':
        return 'hyper'
    else:
        return 'Normal'

# 새로운 컬럼 추가
train['갑상선_인지'] = train.apply(determine_thyroid_status, axis=1)

# test에 갑상선저하인지, 갑상선항진증인지 합쳐서 [정상,갑상선비정상] 으로 나누기
def determine_thyroid_status(row):
    if row['갑상선저하_인지_여부'] == '예':
        return 'hypo'
    elif row['갑상선항진증_인지_여부'] == '예':
        return 'hyper'
    else:
        return 'Normal'

# 새로운 컬럼 추가
test['갑상선_인지'] = test.apply(determine_thyroid_status, axis=1)

In [74]:
train['갑상선_기능_지수'] = train['TSH'] * train['FreeT3'] / train['FreeT4']
test['갑상선_기능_지수'] = test['TSH'] * test['FreeT3'] / test['FreeT4']

In [75]:
train.columns

Index(['ID', '나이', '성별', '티록신_복용_여부', '항갑상선제_복용_여부', '지병_여부', '임신_여부',
       '갑상선_수술_이력', 'I131_치료_여부', '갑상선저하_인지_여부', '갑상선항진증_인지_여부', '리튬_치료_여부',
       '갑상선종_여부', '종양_여부', 'TSH', 'FreeT3', 'FreeT4', 'target', 'TSH범위',
       '갑상선_인지', '갑상선_기능_지수'],
      dtype='object')

# 특성 공학(Feature Engineering)
- Feature Engineering 과정에서 평가를 위한 예측을 해야하기 때문에 테스트 세트에 대해서도 동일한 작업을 진행해줘야 한다.


## Feature Extraction

- 특성으로 사용할 변수 추가하기

In [76]:
train_ft = train.drop(columns=['ID','target'])
test_ft = test.drop(columns=['ID'])
train_ft.shape, test_ft.shape

((4223, 19), (3456, 19))

## Feature Encoding

- One-Hot Encoding

In [77]:
from sklearn.preprocessing import OneHotEncoder
cols = train_ft.select_dtypes("object").columns.tolist()
enc = OneHotEncoder(handle_unknown = 'ignore')

In [78]:
# 학습데이터
tmp = pd.DataFrame(
    enc.fit_transform(train_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
train_ft = pd.concat([train_ft,tmp],axis=1).drop(columns=cols) # 범주형 컬럼 제거
train_ft.head()

,나이,TSH,FreeT3,FreeT4,갑상선_기능_지수,성별_UNK,성별_남,성별_여,티록신_복용_여부_아니오,티록신_복용_여부_예,...,리튬_치료_여부_예,갑상선종_여부_아니오,갑상선종_여부_예,종양_여부_아니오,종양_여부_예,TSH범위_High,TSH범위_low,갑상선_인지_Normal,갑상선_인지_hyper,갑상선_인지_hypo
0,59.0,4.953486,2.021445,0.770000,13.004158,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,63.0,33.000000,1.500000,0.965153,51.287201,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
2,65.0,1.700000,2.300000,0.950000,4.115789,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
3,33.0,6.200000,2.021445,0.660000,18.989335,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
4,64.0,1.200000,2.021445,0.950000,2.553405,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0


In [79]:
# 테스트데이터
tmp = pd.DataFrame(
    enc.transform(test_ft[cols]).toarray(), # 테스트데이터는 transform 만 해야한다.
    columns = enc.get_feature_names_out()
)
test_ft = pd.concat([test_ft,tmp],axis=1).drop(columns=cols) # 범주형 컬럼 제거
test_ft.head()

,나이,TSH,FreeT3,FreeT4,갑상선_기능_지수,성별_UNK,성별_남,성별_여,티록신_복용_여부_아니오,티록신_복용_여부_예,...,리튬_치료_여부_예,갑상선종_여부_아니오,갑상선종_여부_예,종양_여부_아니오,종양_여부_예,TSH범위_High,TSH범위_low,갑상선_인지_Normal,갑상선_인지_hyper,갑상선_인지_hypo
0,37.0,4.953486,2.021445,0.83,12.064099,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,55.0,0.050000,1.500000,0.72,0.104167,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
2,71.0,0.100000,1.900000,0.97,0.195876,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
3,35.0,0.970000,2.021445,0.97,2.021445,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
4,16.0,0.300000,2.021445,1.06,0.572107,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0


## Feature Scaling
- Min-Max Scaling
    - 데이터 수치의 범위를 바꿔주는 정규화 스케일링 기법
    - 데이터 수치의 범위를 0 ~ 1 사이로 바꿔준다.

In [80]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [81]:
# 학습데이터
train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
train_ft.head()

,나이,TSH,FreeT3,FreeT4,갑상선_기능_지수,성별_UNK,성별_남,성별_여,티록신_복용_여부_아니오,티록신_복용_여부_예,...,리튬_치료_여부_예,갑상선종_여부_아니오,갑상선종_여부_예,종양_여부_아니오,종양_여부_예,TSH범위_High,TSH범위_low,갑상선_인지_Normal,갑상선_인지_hyper,갑상선_인지_hypo
0,0.604167,0.010484,0.109830,0.278075,0.014894,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.645833,0.069905,0.080780,0.382435,0.058766,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.666667,0.003591,0.125348,0.374332,0.004709,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
3,0.333333,0.013125,0.109830,0.219251,0.021753,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.656250,0.002532,0.109830,0.374332,0.002918,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0


In [82]:
# 테스트 데이터
test_ft[test_ft.columns] = scaler.transform(test_ft)  # 테스트데이터는 transform 만 해야한다.
test_ft.head()

,나이,TSH,FreeT3,FreeT4,갑상선_기능_지수,성별_UNK,성별_남,성별_여,티록신_복용_여부_아니오,티록신_복용_여부_예,...,리튬_치료_여부_예,갑상선종_여부_아니오,갑상선종_여부_예,종양_여부_아니오,종양_여부_예,TSH범위_High,TSH범위_low,갑상선_인지_Normal,갑상선_인지_hyper,갑상선_인지_hypo
0,0.375000,0.010484,0.109830,0.310160,0.013817,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.562500,0.000095,0.080780,0.251337,0.000111,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
2,0.729167,0.000201,0.103064,0.385027,0.000216,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
3,0.354167,0.002045,0.109830,0.385027,0.002308,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
4,0.156250,0.000625,0.109830,0.433155,0.000648,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0


- 정답 데이터

In [83]:
target = train["target"]
target

0       0
1       1
2       0
3       0
4       0
       ..
4218    0
4219    0
4220    0
4221    0
4222    0
Name: target, Length: 4223, dtype: int64

# 모델 학습및 교차 검증

- catboost 설치

In [84]:
# !pip install catboost

In [85]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=5,shuffle=True, random_state=SEED)

## Voting 방식 앙상블 CV 점수 확인해보기

In [86]:
# 별칭과 모델 객체를 튜플에 담아서 리스트에 넣음
estimators = [

              ("rf", RandomForestClassifier(random_state=SEED)),
              ("ada", AdaBoostClassifier(random_state=SEED)),
              ("gb", GradientBoostingClassifier(random_state=SEED)),
              ("xgb", XGBClassifier(random_state=SEED)),
              ("lgb", LGBMClassifier(random_state=SEED)),
              ("cat", CatBoostClassifier(random_state=SEED)),
]

In [87]:
# VotingClassifier 전달할 아규먼트
params_voting ={
    "estimators" : estimators, # 모델 담은 리스트 전달
    "voting" : "soft" , # soft 또는 hard 전달하여 cv 점수 확인해보기
    "n_jobs" : -1
}

model = VotingClassifier(**params_voting)
scores = cross_val_score(model,train_ft,target,cv = cv ,scoring='f1',n_jobs = -1)
np.mean(scores) # cv 평균 점수

0.8700627278469064

## Stacking 방식 앙상블 CV 점수 확인해보기

In [88]:
# from sklearn.ensemble import StackingClassifier
# from sklearn.linear_model import LogisticRegression

# # StackingClassifier 전달할 아규먼트
# params_stacking ={
#     "estimators" : estimators, # 모델 담은 리스트 전달
#     "final_estimator" : LogisticRegression(random_state=SEED) , # 메타 모델 변경해보기(가벼운 모델 추천)
#     "cv" : cv, # cv 객체
#     "n_jobs" : -1
# }

# model = StackingClassifier(**params_stacking)
# scores = cross_val_score(model,train_ft,target,cv = cv ,scoring='f1',n_jobs = -1)
# np.mean(scores) # cv 평균 점수

# 모델 학습
- 이전 단계의 검증 결과가 유의미한 성능을 보인다면 학습데이터 전체를 학습하고 평가를 위해 테스트 데이터에 대한 예측을 진행한다.

In [89]:
model = VotingClassifier(**params_voting) # Voting 방식 앙상블 모델 학습
model.fit(train_ft,target)

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('ada', AdaBoostClassifier(random_state=42)),
                             ('gb',
                              GradientBoostingClassifier(random_state=42)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_c...
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...)),
                             ('lgb', LGBMClassifier(random_state=42)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x000001D5E62FD090>)],
                 n_jobs=-1, voting='soft')

# 테스트 데이터 예측


In [90]:
pred = model.predict(test_ft)
pred

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

# 평가를 위한 제출 파일 생성
- sample_submission.csv 파일을 불러와서 예측 결과를 target 컬럼에 넣어 csv 파일로 저장후에 컴피티션 페이지에 제출한다.

In [91]:
submit = pd.read_csv(f"{DATA_PATH}sample_submission.csv")
submit

,ID,target
0,test_0,NaN
1,test_1,NaN
2,test_2,NaN
3,test_3,NaN
4,test_4,NaN
...,...,...
3451,test_3451,NaN
3452,test_3452,NaN
3453,test_3453,NaN
3454,test_3454,NaN


In [92]:
submit["target"] = pred
submit

,ID,target
0,test_0,0
1,test_1,0
2,test_2,0
3,test_3,0
4,test_4,0
...,...,...
3451,test_3451,0
3452,test_3452,0
3453,test_3453,1
3454,test_3454,0


- 예측 결과를 csv 파일로 저장하고 컴피티션 페이지에 제출하여 결과 확인하기


In [93]:
submit.to_csv("submit_임태균_0625_03.csv",index=False) # 인덱스는 제외하기 위해 False